In [1]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import(ModelSpec as MS,
                        summarize,
                        poly)
from sklearn.model_selection import train_test_split
from functools import partial
from sklearn.model_selection import \
    (cross_validate,
     KFold, 
     ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

# The Validation Set Approach

We are using the `Auto` dataset which has 392 observations. Using the Validation Set approach, we split into two equal sets of size 196, representing the training and test set.

It is generally a good idea to set a random seed when performing operations like this that contain an element of randomness, so that the results obtained can be reproduced precisely at a later later.

In [2]:
Auto = load_data("Auto")
Auto_train, Auto_valid = train_test_split(Auto, 
                                          test_size=196,
                                          random_state=0)

hp_mm = MS(["horsepower"])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train["mpg"]
model = sm.OLS(y_train, X_train)
results = model.fit()

Hence our estimate for the validation MSE of the linear regression fit is 23.62

In [3]:
X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)
np.mean((y_valid - valid_pred)**2)

23.61661706966988

The validation set approach can also be used to estimate the validation error for higher-degree polynomial regressions.

In [4]:
def evalMSE(
        terms,
        response, 
        train,
        test):
    mm = MS(terms)
    X_train = mm.fit_transform(train) 
    y_train = train[response]
    
    X_test = mm.transform(test)
    y_test = test[response]
    results = sm.OLS(y_train, X_train).fit()
    test_pred = results.predict(X_test)
    
    return np.mean((y_test - test_pred)**2)

The code below generates the array showing the MSE of the linear regression fit with increasing polynomial degree.

In [5]:
MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],
                       'mpg',
                        Auto_train,
                        Auto_valid)
MSE

array([23.61661707, 18.76303135, 18.79694163])

One problem with using the validation set approach is that the MSE results are highly variable and dependent on how the observations are split into the training and validation set. This can be shown below when we run the same code but with a different random seed.

In [6]:
Auto_train, Auto_valid = train_test_split(Auto,
                                          test_size=196,
                                          random_state=3)
MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)], 
                       'mpg',
                       Auto_train,
                       Auto_valid)
MSE

array([20.75540796, 16.94510676, 16.97437833])

# Cross-Validation

In theory, the cross-validation estimate can be computed for any generalized linear model. In practice, however, the simplest way to cross-validation in Python is to use `sklearn`, which has a different interface or API than `statsmodels`, the code we have been using to fit GLMs. We can use a wrapper `sklearn_sm()` to enable us to easily use the cross-validation tools of `sklearn` with models fit by `statsmodels`.

The arguments to `cross_validate()` are as follows: an object with the appropriate `fit()`, `predict()` and `score()` methods, an array of features `X` and a response `Y`. We also included an additional argument `cv` to `cross_validate()`; specifying an integer *K* results in *K*-fold cross-validation. In the code below, `cv` is provided a value corresponding to the total number of observations, which results in leave-one-out cross validation (LOOCV). The `cross-validate()` function produces a dictionary with several components.

In [7]:
hp_model = sklearn_sm(sm.OLS, MS(['horsepower']))
X, Y = Auto.drop(columns=['mpg']), Auto['mpg']
cv_results = cross_validate(hp_model,
                            X,
                            Y,
                            cv=Auto.shape[0]) 
cv_err = np.mean(cv_results['test_score'])
cv_err

24.23151351792924

Just like for the validation set, we can check the MSE for increasingly complex polynomial fits.

In [8]:
cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
M = sklearn_sm(sm.OLS)
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M,
                          X, 
                          Y, 
                          cv=Auto.shape[0])
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.23151352, 19.24821312, 19.33498406, 19.42443033, 19.03323827])

In the `for` loop, there is a given polynomial degree `d`, and so `np.power.outer()` takes the column vector `H`, raise elements in the vector to a power of 0 initially and repeat the process for increasing degrees until degree `d` is reached (inclusive).

The above codes perform LOOCV, but by specifying $K<N$, we can perform *K*-Fold Cross-Validation. The code is similar and is significantly faster.

In [9]:
cv_error = np.zeros(5) 
cv = KFold(n_splits=10,
           shuffle=True,
           random_state=0) # use same splits for each degree for i, d in enumerate(range(1,6)):
for i, d in enumerate(range(1,6)):
        X = np.power.outer(H, np.arange(d+1))
        M_CV = cross_validate(M,
                            X,
                            Y,
                            cv=cv)
        cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.20766449, 19.18533142, 19.27626666, 19.47848403, 19.13720581])

The `cross_validate()` function is flexbile and can take different splitting mechanisms as an argument.

In [10]:
validation = ShuffleSplit(n_splits=1,
                          test_size=196,
                          random_state=0)
results = cross_validate(hp_model,
                         Auto.drop(['mpg'], axis=1), 
                         Auto['mpg'],
                         cv=validation)
results["test_score"]

array([23.61661707])

One can estimate the variability in the test error by running the following:

Note: This standard deviation is not a valid estimate of the sampling variability of the mean test score or the individual scores, since the randomly-selected training samples overlap and hence introduce tracking of residuals and correlations.

In [11]:
validation = ShuffleSplit(n_splits=10, 
                          test_size=196,
                          random_state=0) 
results = cross_validate(hp_model,
                         Auto.drop(['mpg'], axis=1),
                         Auto['mpg'],
                         cv=validation)
results['test_score'].mean(), results['test_score'].std()

(23.802232661034168, 1.421845094109185)

# The Bootstrap

One of the great advantages of the bootstrap approach is that it can be applied in almost all situations.

For the `Portfolio` dataset, the goal is to estimate the sampling variance of the parameter $\alpha$ given in the formula: $$\hat{\alpha} = \frac{\hat{\sigma}_Y^2 - \hat{\sigma}_{XY}}{\hat{\sigma}_X^2 + \hat{\sigma}_Y^2 - 2\hat{\sigma}_{XY}}$$

We can do so using Bootstrap.


In [12]:
Portfolio = load_data('Portfolio')
def alpha_func(D, idx):
    cov_ = np.cov(D[['X','Y']].loc[idx], rowvar=False) 
    return ((cov_[1,1] - cov_[0,1]) /
    (cov_[0,0]+cov_[1,1]-2*cov_[0,1]))

Next we randomly select 100 observations with replacement. This is equivalent to constructing a new bootstrap data set and recomputing $\hat{\alpha}$ based on the new data set.

In [13]:
rng = np.random.default_rng(0) 
alpha_func(Portfolio,
           rng.choice(100,
                      100,
                      replace=True))

0.6074452469619004

This process can be generalised to create a simple function `boot_SE()` for computing the bootstrap standard error for arbitrary functions that take only a data frame as an argument.

In [14]:
def boot_SE(func, 
            D,
            n=None,
            B=1000,
            seed=0):
    rng = np.random.default_rng(seed) 
    first_ , second_ = 0, 0
    n = n or D.shape[0]
    for _ in range(B):
        idx = rng.choice(D.index,
                         n,
                         replace=True) 
        value = func(D, idx)
        first_ += value
        second_ += value**2
    return np.sqrt(second_ / B - (first_ / B)**2)

In [15]:
alpha_SE = boot_SE(alpha_func,
                   Portfolio,
                   B=1000,
                   seed=0)
alpha_SE

0.09118176521277699